In [1]:
#This loops through the songs from the scraped Billboard data, 
#calls the Spotify API twice to gather the metadata for the songs, 
#and adds them to a json.

import json
from pprint import pprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

with open('billboard[2000:3040].json', "r") as f:
    data = json.load(f)

passList = []
failList = []


x = 462                                                # 'just change this to 500 and add another 500 to the x'

while x < 1040:   # do 1000 at a a time in case something breaks
    j = 1
    while j < 10:
        art = data[x]['0'][j]['Artist']
        song = data[x]['0'][j]['Song']
        #print(song + ':    ' + art)
    


        # Make your own Spotify app at https://beta.developer.spotify.com/dashboard/applications
        client_id = 'your client id here'
        client_secret = 'your client secret here'
        title = song
        artist = art

        client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
        sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
        sp.trace=False
        search_querry = title + ' ' + artist
        result = sp.search(search_querry)
        for i in result['tracks']['items']:
            # Find a song that matches title and artist
            if (i['artists'][0]['name'] == artist) and (i['name'] == title):
                track = i['uri']
                track_name = i['name']
                #print (track)
                #print(track_name)
                artist = i['artists'][0]['uri']
                #print (artist)
                break
            else:
                try:
                    # Just take the first song returned by the search (might be named differently)
                    #print (result['tracks']['items'][0]['uri'])
                    print('try    ' + song)
                except:

                    # No results for artist and title
                    print ("Cannot Find URI")
                    print('except    + song')


        try:
            client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
            sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
            sp.trace=False
            artist_features = sp.artist(artist)
            track_features = sp.audio_features(track)


            key = ''
            numkey = track_features[0]['key']
            if numkey == 1:
                key = 'C#'
            elif numkey == 2: 
                key = 'D'
            elif numkey == 3: 
                key = 'D#'
            elif numkey == 4: 
                key = 'E'    
            elif numkey == 5: 
                key = 'F'
            elif numkey == 6: 
                key = 'F#'    
            elif numkey == 7: 
                key = 'G'    
            elif numkey == 8: 
                key = 'G#'    
            elif numkey == 9: 
                key = 'A'    
            elif numkey == 10: 
                key = 'A#'
            elif numkey == 11: 
                key = 'B'    
            else: 
                key = 'C'    

            numMode = track_features[0]['mode']

            mode = 'Major'    
            if numMode == 0: 
                mode = 'Minor'

            durMs = track_features[0]['duration_ms']  
            length = (durMs/1000)/60

#             data[x]['0'][j]['Artist']
#             data[0]['0'][1]['Genres'] = "Pop Punk"    template
            
            data[x]['0'][j]['Artist'] = artist_features['name']
            data[x]['0'][j]['Followers'] = int(artist_features['followers']['total'])
            data[x]['0'][j]['Genres'] = artist_features['genres']
            data[x]['0'][j]['Popularity'] = int(artist_features['popularity'])
            data[x]['0'][j]['Track Name'] = i['name']
            data[x]['0'][j]['KeySig'] = (key + ' ' + mode)
            data[x]['0'][j]['Tempo'] = int(track_features[0]['tempo'])
            data[x]['0'][j]['Length'] = float(length)
            data[x]['0'][j]['TimeSig'] = int(track_features[0]['time_signature'])
            data[x]['0'][j]['Danceability'] = float(track_features[0]['danceability'])
            data[x]['0'][j]['Energy'] = float(track_features[0]['energy'])
            data[x]['0'][j]['Valence'] = float(track_features[0]['valence'])
            data[x]['0'][j]['Acousticness'] = float(track_features[0]['acousticness'])
            data[x]['0'][j]['Key'] = int(numkey)
            data[x]['0'][j]['Mode'] = int(track_features[0]['mode'])
            data[x]['0'][j]['Duration(ms)'] = float(track_features[0]['duration_ms'])
                
            #print(song + '    success')
            passList.append(song)
            #print(result)
            results.append(result)
            with open("billboard[2000:3040].json", "w") as jsonFile:
                json.dump(data, jsonFile)
            
        except Exception:
            #print(song + '    fail')
            failList.append(song)
            pass

        j +=1
    x += 1
    print(x)
    #time.sleep(2)  #just let it rest for a few secs between each week so it doesn't seem as suspicious. idk
print('complete')
print('PASS')
print(len(passList))
print('FAIL')
print(len(failList))

In [2]:
with open('Billboard.json', "r") as f:
    data = json.load(f)